In [ ]:
#sns.violinplot(data=bk_P[['system_size_DC']])
#sns.violinplot(data=bk_P_2[['system_size_DC']])
#sns.violinplot(data=bk_P_3[['system_size_DC']])
#sns.lineplot(data=costs_df, x="install_month", y="cost_per_kW")
#sns.lineplot(data=costs_df2, x="install_month", y="cost_per_kW")
#sns.lineplot(data=rebate_df, x="install_month", y="rebate_or_grant")
# with open('../data/sub_25.p', 'rb') as fp:
#     sub_25_map = pickle.load(fp)
# zip_cluster_df_full['install_count'] = zip_cluster_df_full.zipcode.map(sub_25_map)
# zip_cluster_df_full['percent_homes'] = zip_cluster_df_full.install_count.values/zip_cluster_df_full.home_count.values
# zip_cluster_df_full.head()

# fig = px.choropleth_mapbox(zip_cluster_df_full, geojson=zips, locations='zipcode', color='percent_homes',
#                            color_continuous_scale="Viridis",
#                            mapbox_style="carto-positron",
#                            zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
#                            opacity=0.5,
#                            labels={'system_size_DC':'kW'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
# zc_pair = zip_cluster_df_full[['med_income', 
#                                'med_home_value', 
#                                'pop_density', 
#                                'NREL_PSM_2019', 
#                                'cost_electricity', 
#                                'avg_contractor_wage',
#                                'percent_homes']].dropna()

# #zc_pair['cap_per_home'] = pd.cut(zc_pair.cap_per_home, bins = 5)
# #zc_pair.head()
# sns.pairplot(data = zc_pair)
#from sklearn.linear_model import LinearRegression

# X = zc_pair.drop(columns = 'percent_homes')
# y = zc_pair.percent_homes

# model = LinearRegression()
# # fit the model
# model.fit(X, y)
# # get importance
# importance = model.coef_
# importance

# zc_pair['prediction'] = model.predict(X)
# zc_pair['performance_vs_expectation'] = (zc_pair.percent_homes - zc_pair.prediction) / zc_pair.prediction
# zc_pair['zipcode'] = zip_cluster_df_full.zipcode

# zc_pair = zc_pair[~zc_pair['performance_vs_expectation'].isin(find_anomalies(zc_pair['performance_vs_expectation']))]
# zc_pair.head()

# fig = px.choropleth_mapbox(zc_pair, geojson=zips, locations='zipcode', color='performance_vs_expectation',
#                            color_continuous_scale="Viridis",
#                            mapbox_style="carto-positron",
#                            zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
#                            opacity=0.5,
#                            labels={'system_size_DC':'kW'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

# import pickle

# with open('../data/earliest_dates.p', 'rb') as fp:
#     earliest_dict = pickle.load(fp)

# plt.clf()
# fig, axes = plt.subplots(25,figsize = (12, 120))
# min_yr = pd.to_datetime('1998-04-30 00:00:00').year
# min_mo = pd.to_datetime('1998-04-30 00:00:00').month




# for i, key in enumerate(earliest_dict.keys()):
#     eval_df =  bk_P_rt[bk_P_rt.zip_code.isin(earliest_dict[key].keys())]
#     eval_df['p_mo'] = eval_df.zip_code.map(earliest_dict[key])
    
#     eval_df['num_months'] = eval_df.install_month.apply(
#         lambda d: (d.year - min_yr) * 12 + (d.month - min_mo)
#     )
    
#     eval_df['program_months'] = eval_df.num_months - eval_df.p_mo
    
#     eval_df['total_cost'] = eval_df.total_installed_price - eval_df.rebate_or_grant
#     #eval_df['prog_date'] = pd.to_datetime(eval_df.zip_code.map(program_dict[key]))
#     #eval_df['num_months'] = (eval_df.install_month.dt.year - eval_df['prog_date'].dt.year) * 12 + (eval_df.install_month.dt.month - eval_df['prog_date'].dt.month)
    
#     plotter = eval_df.groupby(['p_mo','program_months']).total_cost.count().reset_index()
#     #display(plotter.sort_values(by = 'num_months').head(20))
#     #plotter['prog_date'] = plotter['prog_date'].apply(lambda c: str(c))
#     maxes = plotter.groupby(['p_mo']).total_cost.max().reset_index()
#     max_dict = dict(zip(maxes.p_mo,maxes.total_cost))
#     plotter['max_val'] = plotter.p_mo.map(max_dict)
#     plotter['percent_max'] = plotter.total_cost / plotter.max_val
#     plotter_2 = plotter.groupby('program_months').percent_max.mean().reset_index()
#     sns.lineplot(ax=axes[i], data=plotter_2, x='program_months', y='percent_max')  
#     axes[i].set_title("program_id_{}".format(str(key)))

# zip_cluster_df_full['install_level'] = pd.qcut(zip_cluster_df_full.percent_homes, 7,
#                                               labels=[0,1,2,3,4,5,6]) #"low", "med-low", "med", "med-high","high"
# zip_cluster_df_full = zip_cluster_df_full.dropna()

# scale_pipe = Pipeline(
#     [
        
#         ("standard",StandardScaler()),
#     ]
# )

# df_raw = zip_cluster_df_full.copy()[['housing_density','med_income','NREL_PSM_2019', 'cost_electricity']]
# df_transformed = scale_pipe.fit_transform(df_raw)

# h_scores = {}
# c_scores = {}



# y_WS = zip_cluster_df_full.install_level
# # reduced hyperparameters for runtime savings - derived from wider range
# for i in np.linspace(.2,.5,16):
#     for j in np.linspace(5,50,10):
#         try:
#             dbscan = DBSCAN(eps = i, min_samples = j)
#             cls = dbscan.fit_predict(df_transformed)
#             hs = homogeneity_score(cls, y_WS)
#             cs = completeness_score(cls, y_WS)
#             if len(np.unique(cls)) > 1:
#                 h_scores[(i,j)] = hs
#                 c_scores[(i,j)] = cs
#             else:
#                 h_scores[(i,j)] = 0
#                 c_scores[(i,j)] = 0
#         except:
#             h_scores[(i,j)] = 0
#             c_scores[(i,j)] = 0
# sdh = sorted(h_scores.items(), key=lambda item: item[1], reverse = True)
# sdc = sorted(c_scores.items(), key=lambda item: item[1], reverse = True)
    
    
# sdc

# df_raw = zip_cluster_df_full.copy()[['housing_density','med_income','NREL_PSM_2019', 'cost_electricity']]
# df_transformed = scale_pipe.fit_transform(df_raw)


# i_c = zip_cluster_df_full.install_level

# agg = DBSCAN(eps = .22, min_samples = 5) #AgglomerativeClustering(n_clusters = 11)
# cls_a2 = agg.fit_predict(df_transformed)

# pca_a = df_transformed.copy()
# tsne_a = df_transformed.copy()

# X_pca_a = PCA(n_components = 2, random_state=42).fit_transform(pca_a)
# X_tsne_a = TSNE(n_components = 2, random_state = 42).fit_transform(tsne_a)

# plt.clf()
# fig, axes = plt.subplots(4, figsize=(12,12))

# axes[0].scatter(X_pca_a[:, 0], X_pca_a[:, 1], s=10, c=cls_a2)
# axes[1].scatter(X_tsne_a[:, 0], X_tsne_a[:, 1], s=10, c=cls_a2)
# axes[2].scatter(X_pca_a[:, 0], X_pca_a[:, 1], s=10, c=i_c)
# axes[3].scatter(X_tsne_a[:, 0], X_tsne_a[:, 1], s=10, c=i_c)



# axes[0].set_title("PCA_Agglomerative")
# axes[1].set_title("tsne_Agglomerative")
# axes[0].set_title("PCA_lvl")
# axes[1].set_title("tsne_lvl")
        
# cs = completeness_score(cls_a2, i_c)
# print(cs)